In [ ]:
# Setup
%pip install databricks-sdk==0.40.0 mlflow==2.18.0 --quiet
dbutils.library.restartPython()

In [ ]:
# Initialize progress tracking
%run ../_resources/00-setup $reset_all_data=false

import sys
sys.path.append('../_resources')
from gamification_framework import (
    init_learner, 
    display_challenge_intro,
    display_challenge_success,
    display_badge_awarded
)

learner = init_learner()
display_challenge_intro(
    challenge_name="Prompt Engineering Basics",
    difficulty="Beginner",
    points=100,
    description="Learn to write effective prompts that produce reliable, high-quality AI responses for turbine maintenance scenarios."
)

---

## 📚 Lesson 1: The Bad Prompt Problem

Here's what a junior engineer wrote. Let's see why it doesn't work:

In [ ]:
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()

# ❌ BAD PROMPT EXAMPLE
bad_prompt = """
Help with turbine.
"""

# Test it
response = w.serving_endpoints.query(
    name="databricks-meta-llama-3-1-70b-instruct",
    messages=[
        {"role": "system", "content": bad_prompt},
        {"role": "user", "content": "Turbine making noise"}
    ],
    max_tokens=150
)

print("🔴 BAD PROMPT OUTPUT:")
print(response.choices[0].message.content)
print("\n❌ Problems:")
print("  - Too vague")
print("  - No context about turbines")
print("  - No guidance on response format")
print("  - No safety considerations")

---

## ✅ Lesson 2: The Good Prompt Pattern

Now let's apply prompt engineering principles:

### The RACE Framework

- **R**ole: Define who the AI is
- **A**ction: Specify what it should do  
- **C**ontext: Provide relevant background
- **E**xpectation: Describe the desired output

In [ ]:
# ✅ GOOD PROMPT EXAMPLE
good_prompt = """
You are an expert wind turbine maintenance assistant with 10+ years of field experience.

Your role is to help junior technicians diagnose and resolve turbine issues quickly and safely.

Context:
- We maintain GE 2.5MW turbines at offshore wind farms
- Technicians have basic training but need expert guidance
- Safety is the top priority
- Minimize downtime to maintain power generation

When responding:
1. Start with safety considerations
2. Provide a clear diagnosis
3. Give step-by-step troubleshooting instructions
4. Include estimated time and parts needed
5. Escalate to senior engineer if issue is critical

Format responses as:
🛡️ SAFETY: [safety steps]
🔍 DIAGNOSIS: [likely cause]
🔧 STEPS: [numbered steps]
⏱️ ESTIMATED TIME: [time]
📦 PARTS NEEDED: [list]
⚠️ ESCALATE IF: [conditions]
"""

response = w.serving_endpoints.query(
    name="databricks-meta-llama-3-1-70b-instruct",
    messages=[
        {"role": "system", "content": good_prompt},
        {"role": "user", "content": "Turbine making grinding noise from gearbox area"}
    ],
    max_tokens=500
)

print("✅ GOOD PROMPT OUTPUT:")
print(response.choices[0].message.content)
print("\n✅ Much better because:")
print("  - Clear role and expertise level")
print("  - Specific context about equipment")
print("  - Structured output format")
print("  - Safety-first approach")

---

## 🎯 Challenge 1: Write Your First Prompt

**Your Task:** Create a system prompt for helping technicians interpret sensor data.

**Requirements:**
- Role: Senior Data Analyst specialized in IoT sensor analysis
- Context: Analyzing vibration, temperature, and power output sensors
- Output: Should identify anomalies and suggest next actions
- Include: Confidence levels for predictions

In [ ]:
# 💪 YOUR TURN - Write your prompt here!

your_prompt = """
# TODO: Write your system prompt here using the RACE framework
# 
# Tips:
# - Start with a clear role definition
# - Provide context about sensor types and normal ranges
# - Specify output format
# - Include confidence scoring (High/Medium/Low)
# - Add when to alert senior engineers

"""

# Test your prompt
test_sensor_data = """
Turbine ID: WT-042
Vibration: 12.5 mm/s (normal: <8 mm/s)
Temperature: 85°C (normal: 60-75°C)
Power Output: 2.1 MW (rated: 2.5 MW)
"""

response = w.serving_endpoints.query(
    name="databricks-meta-llama-3-1-70b-instruct",
    messages=[
        {"role": "system", "content": your_prompt},
        {"role": "user", "content": f"Analyze this sensor data:\n{test_sensor_data}"}
    ],
    max_tokens=400
)

print("📊 YOUR PROMPT OUTPUT:")
print(response.choices[0].message.content)

### ✅ Self-Evaluation Checklist

Did your prompt produce a response that:

- [ ] Clearly identifies which sensors are abnormal?
- [ ] Provides confidence levels?
- [ ] Suggests specific next actions?
- [ ] Uses the format you specified?
- [ ] Includes escalation criteria?

If you checked 4-5 boxes: **Excellent!** 🎉  
If you checked 2-3 boxes: **Good start** - refine and rerun  
If you checked 0-1 boxes: Review the example and try again

---

## 📈 Lesson 3: Few-Shot Learning

Sometimes the best way to teach AI is through examples:

In [ ]:
# Few-shot prompt with examples
few_shot_prompt = """
You diagnose turbine error codes and provide maintenance actions.

Examples:

Error Code: ERR-001
Response: ⚠️ Blade Pitch Fault | 🔧 Recalibrate pitch motors | ⏱️ 2 hours | 🔴 Priority: HIGH

Error Code: ERR-015  
Response: ℹ️ Sensor Drift | 🔧 Replace temperature sensor | ⏱️ 30 min | 🟡 Priority: MEDIUM

Error Code: ERR-099
Response: 🚨 EMERGENCY SHUTDOWN | 🔧 DO NOT RESTART | ⏱️ Call Control Center | 🔴 Priority: CRITICAL

Now diagnose the following error code using the same format:
"""

response = w.serving_endpoints.query(
    name="databricks-meta-llama-3-1-70b-instruct",
    messages=[
        {"role": "system", "content": few_shot_prompt},
        {"role": "user", "content": "Error Code: ERR-042"}
    ],
    max_tokens=200
)

print("🎯 Few-Shot Output:")
print(response.choices[0].message.content)
print("\n✨ Notice how it learned the format from examples!")

---

## 🎯 Challenge 2: Create a Few-Shot Prompt

**Your Task:** Create a few-shot prompt to teach AI how to prioritize maintenance work orders.

**Requirements:**
- Provide 3 example work orders with different priorities
- Show how priority is calculated based on: safety risk, downtime impact, and part availability
- Output format: Priority score (1-10), urgency level, and recommended schedule

In [ ]:
# 💪 YOUR TURN - Create your few-shot prompt

your_few_shot_prompt = """
# TODO: Create a few-shot learning prompt for work order prioritization
#
# Include 3 examples showing:
# Example 1: High priority (safety issue)
# Example 2: Medium priority (performance issue)
# Example 3: Low priority (preventive maintenance)
#
# Each example should show:
# - Work Order description
# - Your prioritization logic  
# - Priority score (1-10)
# - Urgency level (CRITICAL/HIGH/MEDIUM/LOW)
# - Recommended schedule

"""

# Test with a new work order
test_work_order = """
Work Order #WO-234:
Issue: Gearbox oil level 15% below minimum
Current Status: Turbine operational but efficiency reduced by 8%
Parts Available: Yes
Safety Risk: Low (no immediate danger)
Impact: $1,200/day revenue loss
"""

response = w.serving_endpoints.query(
    name="databricks-meta-llama-3-1-70b-instruct",
    messages=[
        {"role": "system", "content": your_few_shot_prompt},
        {"role": "user", "content": f"Prioritize this work order:\n{test_work_order}"}
    ],
    max_tokens=300
)

print("🎯 YOUR FEW-SHOT OUTPUT:")
print(response.choices[0].message.content)

---

## 📊 Challenge Validation & Scoring

Let's evaluate your prompts and award points!

In [ ]:
# Automatic quality evaluation
def evaluate_prompt_quality(prompt_text: str, test_output: str) -> dict:
    """Evaluate prompt quality based on key criteria"""
    score = 0
    feedback = []
    
    # Check prompt structure
    if len(prompt_text) > 200:
        score += 20
        feedback.append("✅ Prompt has good detail (+20)")
    else:
        feedback.append("⚠️ Prompt could be more detailed")
    
    # Check for role definition
    role_keywords = ['you are', 'expert', 'specialist', 'assistant']
    if any(keyword in prompt_text.lower() for keyword in role_keywords):
        score += 20
        feedback.append("✅ Clear role definition (+20)")
    else:
        feedback.append("⚠️ Missing clear role definition")
    
    # Check for context
    if 'context' in prompt_text.lower() or len(prompt_text.split('\n')) > 5:
        score += 20
        feedback.append("✅ Good context provided (+20)")
    else:
        feedback.append("⚠️ Could use more context")
    
    # Check output has structure
    if len(test_output) > 100 and any(char in test_output for char in [':', '-', '•', '1.', '2.']):
        score += 20
        feedback.append("✅ Output is well-structured (+20)")
    else:
        feedback.append("⚠️ Output could be more structured")
    
    # Check for safety/priority keywords
    important_concepts = ['safety', 'priority', 'critical', 'urgent', 'escalate']
    if any(concept in prompt_text.lower() for concept in important_concepts):
        score += 20
        feedback.append("✅ Includes important considerations (+20)")
    else:
        feedback.append("⚠️ Missing key safety/priority considerations")
    
    return {"score": score, "feedback": feedback}

# Evaluate Challenge 1
print("📊 CHALLENGE 1 EVALUATION")
print("=" * 50)
eval1 = evaluate_prompt_quality(your_prompt, response.choices[0].message.content)
print(f"\nScore: {eval1['score']}/100")
for item in eval1['feedback']:
    print(f"  {item}")

# Award points and badge if earned
if eval1['score'] >= 80:
    learner.complete_challenge("prompt_engineering_basics_ch1", points=50)
    print("\n🎉 Challenge 1 Passed! (+50 points)")
    
    if eval1['score'] >= 90:
        learner.award_badge("prompt_master")
        display_badge_awarded("prompt_master")
else:
    print("\n💡 Try improving your prompt and re-run this cell!")

---

## 🎓 Key Takeaways

You've learned:

✅ The **RACE framework** for prompt engineering  
✅ How to write **specific, structured prompts**  
✅ **Few-shot learning** to teach through examples  
✅ **Quality evaluation** of prompt outputs

### 📚 Further Reading

- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/prompt-engineering)
- [Databricks Prompt Engineering](https://docs.databricks.com/en/generative-ai/prompt-engineering.html)

---

## 🚀 Next Steps

Ready for more? Continue to:

- **05.1.2-prompt-optimization-challenge**: Advanced prompt patterns
- **05.1.3-few-shot-learning**: Deep dive into learning from examples

Or jump ahead to build your first AI agent in **05.2-agent-creation-guide**!

In [ ]:
# Check your overall progress
learner.display_progress()